#### Setting environments

In [2]:
import torch, os, gc
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [ ]:
# Set options
embed_ver = ["clstm", "esm2", "bert", "t5"]
data_path = "../data/test_exam/"
model_path = "../models/classifier_indiv/"
result_path = "../results/"

In [ ]:
col_str = ['file_id', 'organism', 'locus_tag', 'ess']
batch_size = 256

In [4]:
# Set data list for test dataset
ts_data = {
    "data1": ["C018"],  # "Escherichia coli K-12 BW25113"
    "data2": ["C016"],  # "Escherichia coli K-12 MG1655"
    "data3": ["O046"],  # "synthetic bacterium JCVI-Syn3A"
    "data4": ["C048"],  # Bacteroides thetaiotaomicron VPI-5482
    "data5": ["C050"]  # Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S
}

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Define function to record perfomance result
def record_perform(emb_ver, file_id, organ, y_real, y_conf, y_prd):
    y_real = y_real.cpu().numpy()
    y_conf = y_conf.cpu().numpy()
    y_prd = y_prd.cpu().numpy()
    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "emb": [emb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [3]:
# Set model architecture
class Classifier(nn.Module):
    def __init__(self, input_size, num_layers, unit_decrease):
        super(Classifier, self).__init__()
        layers = [nn.BatchNorm1d(input_size), nn.Dropout(0.5)]
        in_dim = input_size
        out_dim = 1024
        for i in range(num_layers):            
            out_dim = max(2, out_dim // unit_decrease)
            layers.append(nn.Linear(in_dim, out_dim))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            in_dim = out_dim
        layers.append(nn.Linear(out_dim, 1))
        self.cls_block = nn.Sequential(*layers)
        
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        return self.cls_block(x)

#### Evaluate model

In [ ]:
df_eval = pd.DataFrame()

for ver in embed_ver:
    model_name = f'cls-{ver}'
    print(f"\n>>>> {model_name} <<<<")    
    
    # load dataset
    data = pd.read_csv(data_path + f"data_emb-{ver}.csv")
    display("Raw data:", data)

    #### Preprocess for test dataset ####
    col_num = [col for col in data.columns if col not in col_str]

    # get test datasets
    loc_ts = {}
    data_ts = {}
    org_ts = {}
    for ts_ver, ids in ts_data.items():
        # get test sample locations
        loc_ts[ts_ver] = data['file_id'].isin(ids)
        # get test samples
        data_ts[ts_ver] = data[loc_ts[ts_ver]]
        org = []
        # get test organism list
        for i in ids:
            organ = data_ts[ts_ver]['organism'][data_ts[ts_ver]['file_id'] == i].to_list()
            if len(organ) > 0:
                org.append(organ[0])
        org_ts[ts_ver] = org

        print("Test dataset(" + ts_ver + "):", data_ts[ts_ver].shape)
    print("Test organism:", org_ts, len(org_ts))

    # split info.& inputs & labels of the test datasets
    info_ts = {}
    y_ts = {}
    test_loader = {}
    for ts_ver, df in data_ts.items():
        info_ts[ts_ver] = df[col_str]
        X_ts = torch.tensor(df[col_num].astype('float32').values)
        y_ts[ts_ver] = torch.tensor(df['ess'].astype('float32').values)
        print("Splited test dataset(" + ts_ver + "):", X_ts.shape, y_ts[ts_ver].shape)                    
        # generate dataloader by the test datasets
        dataset_ts = TensorDataset(X_ts, y_ts[ts_ver])
        test_loader[ts_ver] = DataLoader(dataset_ts, batch_size=batch_size, shuffle=False)

    # get the total test dataset
    loc_ts_all = [sum(loc) >= 1 for loc in zip(*loc_ts.values())]
    info_ts_all = data.loc[loc_ts_all, col_str]
    X_ts_all = torch.tensor(data.loc[loc_ts_all, col_num].astype('float32').values)
    y_ts_all = torch.tensor(data.loc[loc_ts_all, 'ess'].astype('float32').values)
    
    print("Splited test dataset(all):", X_ts_all.shape, y_ts_all.shape)

    # generate dataloader of total test dataset
    test_all_dataset = TensorDataset(X_ts_all, y_ts_all)
    test_all_loader = DataLoader(test_all_dataset, batch_size=batch_size, shuffle=False)
    

    #### Evaluate model ####
    
    # generate model instance
    model = Classifier(
        input_size=X_ts_all.shape[-1],
        num_layers=3,
        unit_decrease=2
    ).to(device)

    # load model weight
    model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
    model.eval()

    ## model evaluations by test dataset ##
    for ts_ver, ids in ts_data.items():
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader[ts_ver]:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch).squeeze()
                all_preds.append(preds.cpu())
                all_labels.append(y_batch.cpu())
        
        # concatenate results to one tensor
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(all_preds)
        prd_cls = (prd_conf >= 0.5).int()
        # performances by testset
        perform = record_perform(
            emb_ver=ver,
            file_id="+".join(ids),
            organ="+".join(org_ts[ts_ver]),
            y_real=y_ts[ts_ver],
            y_conf=prd_conf,
            y_prd=prd_cls,
        )
        df_eval = pd.concat([df_eval, perform], ignore_index=True)
        print(f"- Test in {ts_ver} was done.")
    
    gc.collect()
    
    ## model evaluation on the total test dataset ##
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in test_all_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(X_batch).squeeze()
            all_preds.append(preds.cpu())
            all_labels.append(y_batch.cpu())
    
    # concatenate results to one tensor
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # convert logits to confidences & classes
    prd_conf = torch.sigmoid(all_preds)
    prd_cls = (prd_conf >= 0.5).int()

    # performances on total testset
    perform = record_perform(
        emb_ver=ver,
        file_id="total",
        organ="all",
        y_real=y_ts_all,
        y_conf=prd_conf,
        y_prd=prd_cls
    )
    df_eval = pd.concat([df_eval, perform], ignore_index=True)
    print(f"- Test in total testset was done.")

    # concatenate the protein info. & predicted confidences
    df_prd = pd.DataFrame(prd_conf, columns=["conf"], index=info_ts_all.index)
    df_prd = pd.concat([info_ts_all, df_prd], axis=1)

    # save the model prediction result
    df_prd.to_csv(f"{result_path}prd-indiv_class/{model_name}.csv", index=False)
    
    print(f"- Prediction by {model_name} was done.")

# save the model perfomance result
display("Model performance:", df_eval)
df_eval.to_csv(f"{result_path}eval-indiv_class.csv", index=False)